In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape) # (842663, 112)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829277, 113)



## Abstract Final Actions


1.  **Create new sample with clear count of responders and provide final number of responders here: The total number of responders in this data set was as follows: <u data-renderer-mark="true">362</u>**

    1.  Exclude training session and aborted session CSV files from dataset

    2.  Exclude CSV files with more than one triage run

In [4]:

# Loop through each unique file and scene combination
time_groups_dict = {}
mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False)
for (file_name), df1 in frvrs_logs_df[mask_series].groupby('file_name'):
    actions_list = []
    
    # Add the scene type for this run
    for (scene_index), df2 in df1.groupby('scene_index'):
        scene_type = df2.scene_type.unique().tolist()
        if len(scene_type) != 1: raise
        else: scene_type = scene_type[0]
        actions_list.append(scene_type)
    
    time_groups_dict[file_name] = actions_list
triage_count_df = pd.DataFrame([{'file_name': k, 'triage_count': v.count('Triage')} for k, v in time_groups_dict.items()])
# display(triage_count_df.groupby('triage_count').size().to_frame().rename(columns={0: 'instances_count'}))

# Infer the number of responders from the files with one triage scene
print('''OSU has cleaned/revised the files. The most common problem was that we previously only had one "Gary" as our training so many of these had a "Gary" and then the triage run. (I am keying on "Mike" to exclude the training sessions.) At any rate, it was easier for them to just delete the "Gary" training session from the files. 

There were a few where OSU could not figure out what was going on, so they just deleted them. It is not worth it to fuss with them since we have plenty of data. (OSU noted these in the spreadsheet.)''')
mask_series = (triage_count_df.triage_count == 1)
responders_list = triage_count_df[mask_series].file_name.tolist()
print(
    '\nExcluding training and aborted runs from the dataset, the total number of responders in this data set after the'
    f' OSU revision of the double-triaged files is now {len(responders_list)}'
    ' (assuming responders are represented in one and only one file with one training run in it).'
)

OSU has cleaned/revised the files. The most common problem was that we previously only had one "Gary" as our training so many of these had a "Gary" and then the triage run. (I am keying on "Mike" to exclude the training sessions.) At any rate, it was easier for them to just delete the "Gary" training session from the files. 

There were a few where OSU could not figure out what was going on, so they just deleted them. It is not worth it to fuss with them since we have plenty of data. (OSU noted these in the spreadsheet.)

Excluding training and aborted runs from the dataset, the total number of responders in this data set after the OSU revision of the double-triaged files is now 389 (assuming responders are represented in one and only one file with one training run in it).


In [5]:

# Store the new feature
new_column_name = 'is_a_one_triage_file'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = False
    mask_series = frvrs_logs_df.file_name.isin(responders_list)
    frvrs_logs_df.loc[mask_series, new_column_name] = True
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 111)
columns_list = ['session_uuid', 'file_name', 'logger_version', 'scene_index', 'scene_type', 'is_scene_aborted', 'is_a_one_triage_file']
display(nu.get_minority_combinations(frvrs_logs_df[columns_list], ['scene_type', 'is_a_one_triage_file']))

,session_uuid,file_name,logger_version,scene_index,scene_type,is_scene_aborted,is_a_one_triage_file
678984,724fdf45-3165-43b8-b9ca-dc07102d2886,v.1.0/Clean 724fdf45-3165-43b8-b9ca-dc07102d28...,1.0,9,Orientation,False,False
774929,d06f4054-097a-46ef-a88e-69665bf42389,v.1.0/clean-d06f4054-097a-46ef-a88e-69665bf423...,1.0,0,Orientation,False,True
291312,aec5d448-c4e6-4af7-8e36-d258c7bb6f96,Disaster Day 2022/JS_1016.csv,1.3,1,Triage,False,False
75907,79426650-c950-40ee-816e-decddddd22e9,All CSV files renamed by date/04.19.23.1310.csv,1.3,1,Triage,False,True


In [6]:

# Save the time spent on the first and last triage of the files with two scenes to Excel
file_path = '../saves/xlsx/double_runs.xlsx'
if not os.path.exists(file_path):
    mask_series = (triage_count_df.triage_count == 2)
    file_names_list = triage_count_df[mask_series].file_name.tolist()
    mask_series = frvrs_logs_df.file_name.isin(file_names_list) & (frvrs_logs_df.scene_type == 'Triage')
    rows_list = []
    for (file_name), df1 in frvrs_logs_df[mask_series].groupby('file_name'):
        min_triage = df1.scene_index.min()
        mask_series = (frvrs_logs_df.scene_index == min_triage) & (frvrs_logs_df.file_name == file_name)
        df = frvrs_logs_df[mask_series]
        min_start = df.elapsed_time.min()
        min_stop = df.elapsed_time.max()
        min_elapsed = min_stop - min_start
        
        max_triage = df1.scene_index.max()
        mask_series = (frvrs_logs_df.scene_index == max_triage) & (frvrs_logs_df.file_name == file_name)
        df = frvrs_logs_df[mask_series]
        max_start = df.elapsed_time.min()
        max_stop = df.elapsed_time.max()
        max_elapsed = max_stop - max_start
    
        row_dict = {}
        row_dict['file_name'] = file_name
        row_dict['min_elapsed'] = min_elapsed
        row_dict['max_elapsed'] = max_elapsed
        rows_list.append(row_dict)
    df = DataFrame(rows_list)
    df['elapsed_diff'] = df.max_elapsed - df.min_elapsed
    df.file_name = df.file_name.map(lambda x: nu.get_new_file_name(x))
    
    # Save the DataFrame to an Excel file with the index
    df.sort_values('elapsed_diff').set_index('file_name').applymap(
        lambda x: nu.format_timedelta(timedelta(milliseconds=int(x))), na_action='ignore'
    ).to_excel(file_path, index=True)


2.  **Take the newly created data set and compute the following statistics:**


    a.  Calculate triage time defined as “time start of session until time end of session”

        Median = 7:38, SD = ±2:27, IQR = (6:13, 8:59)

In [7]:

# Get the run's entire history
mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False) & (frvrs_logs_df.is_a_one_triage_file == True)
groupby_columns = ['session_uuid', 'scene_index']
rows_list = []
for (session_uuid, scene_index), df1 in frvrs_logs_df[mask_series].groupby(groupby_columns):
    row_dict = {}
    for cn in groupby_columns: row_dict[cn] = eval(cn)
    time_start = df1.elapsed_time.min()
    row_dict['time_start'] = time_start
    time_stop = df1.elapsed_time.max()
    row_dict['time_stop'] = time_stop
    row_dict['triage_time'] = time_stop - time_start
    rows_list.append(row_dict)
run_time_df = DataFrame(rows_list)
nu.show_time_statistics(run_time_df, ['triage_time'])

,mean,mode,median,SD,min,25%,50%,75%,max
triage_time,7:36,10:22,7:38,±2:27,11 sec,6:13,7:38,8:59,15:52



    b.  Calculate time to hemorrhage control for scene defined as “time of start scene to time last hemorrhage control (last tourniquet applied)”

        Median = 3:51, SD = ±1:44, IQR = (2:41, 4:52)

In [8]:

tool_columns_list = [
    'tool_applied_attachment_point', 'tool_applied_tool_location', 'tool_applied_data', 'tool_applied_sender', 'tool_applied_attach_message', 'tool_applied_patient_id'
]
print(tool_columns_list)
frvrs_logs_df[tool_columns_list].drop_duplicates()

['tool_applied_attachment_point', 'tool_applied_tool_location', 'tool_applied_data', 'tool_applied_sender', 'tool_applied_attach_message', 'tool_applied_patient_id']


,tool_applied_attachment_point,tool_applied_tool_location,tool_applied_data,tool_applied_sender,tool_applied_attach_message,tool_applied_patient_id
0,NaN,NaN,NaN,NaN,NaN,NaN
51,RChest_Breathe1 (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),right_chest,Needle_Decomp(Clone) (UnityEngine.GameObject),AppliedDecompNeedle,NaN
111,LeftUpLeg (UnityEngine.GameObject),tor_ring (1) (UnityEngine.GameObject),tourniquet(Clone) (UnityEngine.GameObject),AppliedTourniquet,NaN,Mike_1 Root
170,skinCollider_BodyCollideLOD (UnityEngine.GameO...,NaN,NaN,AppliedPackingGauze,NaN,Mike_2 Root
211,skinCollider_BodyCollideLOD (UnityEngine.GameO...,NaN,NaN,AppliedDressingGauze,NaN,Mike_0 Root
...,...,...,...,...,...,...
826348,skinCollider_BodyNudeLOD (UnityEngine.GameObject),null,null,AppliedPackingGauze,NaN,Bob_10 Root
826474,skinCollider_BodyNudeLOD (UnityEngine.GameObject),null,null,AppliedPackingGauze,NaN,Bob_1 Root
826856,RightUpLeg (UnityEngine.GameObject),tor_ring (1) (UnityEngine.GameObject),tourniquet(Clone) (UnityEngine.GameObject),AppliedTourniquet,NaN,Helga_6 Root
827747,RightArm (UnityEngine.GameObject),tor_ring (1) (UnityEngine.GameObject),tourniquet(Clone) (UnityEngine.GameObject),AppliedTourniquet,NaN,Military Mike Jungle Burned_0 Root


In [9]:

injury_columns_list = [
    'injury_record_injury_treated', 'injury_record_injury_treated_with_wrong_treatment', 'injury_treated_injury_treated',
    'injury_treated_injury_treated_with_wrong_treatment'
]#[cn for cn in frvrs_logs_df.columns if 'injury' in cn]
print(injury_columns_list)
frvrs_logs_df[injury_columns_list].drop_duplicates()

['injury_record_injury_treated', 'injury_record_injury_treated_with_wrong_treatment', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment']


,injury_record_injury_treated,injury_record_injury_treated_with_wrong_treatment,injury_treated_injury_treated,injury_treated_injury_treated_with_wrong_treatment
0,NaN,NaN,NaN,NaN
6,False,False,NaN,NaN
50,NaN,NaN,True,False
528,NaN,NaN,False,True
533,NaN,NaN,True,True


In [10]:

columns_list = [cn for cn in frvrs_logs_df.columns if ('injury' in cn) and cn.endswith('_id')]
print(columns_list)
frvrs_logs_df[columns_list].drop_duplicates()

['injury_record_id', 'injury_record_patient_id', 'injury_treated_id', 'injury_treated_patient_id', 'injury_id']


,injury_record_id,injury_record_patient_id,injury_treated_id,injury_treated_patient_id,injury_id
0,NaN,NaN,NaN,NaN,NaN
6,R Chest Collapse,Mike_3 Root,NaN,NaN,R Chest Collapse
8,L Shoulder Puncture,Mike_2 Root,NaN,NaN,L Shoulder Puncture
10,L Thigh Laceration,Mike_1 Root,NaN,NaN,L Thigh Laceration
12,R Forearm Laceration,Mike_0 Root,NaN,NaN,R Forearm Laceration
...,...,...,...,...,...
825921,NaN,NaN,R Side Puncture,Lily_5 Root,R Side Puncture
825988,NaN,NaN,R Thigh Laceration,Lily_5 Root,R Thigh Laceration
826099,NaN,NaN,L Bicep Puncture,Lily_11 Root,L Bicep Puncture
826128,NaN,NaN,L Forearm Laceration,Lily_11 Root,L Forearm Laceration


In [15]:

# Get the total and bleeder scene count
base_mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False) & (frvrs_logs_df.is_a_one_triage_file == True)
mask_series = base_mask_series & frvrs_logs_df.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
groupby_columns = ['session_uuid', 'scene_index']
total_scene_count = len(frvrs_logs_df[base_mask_series].groupby(groupby_columns).groups)
gb = frvrs_logs_df[mask_series].groupby(groupby_columns)
bleeder_scene_count = len(gb.groups)

# Iterate through each patient in each run in each file
rows_list = []
# action_types_set = set()
for (session_uuid, scene_index), injury_treated_df in gb:
    row_dict = {}
    for cn in groupby_columns: row_dict[cn] = eval(cn)
    
    # Get the entire history of this scene for the run start
    mask_series = True
    for cn in groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
    scene_df = frvrs_logs_df[mask_series]
    time_start = scene_df.elapsed_time.min()
    row_dict['scene_start'] = time_start
    
    for patient_id, patient_df in scene_df.groupby('patient_id'):
        row_dict['patient_id'] = patient_id
        
        # Is this patient bleeding?
        bleeding_mask_series = patient_df.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
        if patient_df[bleeding_mask_series].shape[0]:
            row_dict['is_patient_bleeding'] = True
            
            # Get the time of patient engagement
            mask_series = patient_df.action_type.isin(['PATIENT_ENGAGED'])
            patient_engaged = patient_df[mask_series].elapsed_time.min()
            row_dict['patient_engagement_time'] = patient_engaged
            
            # Get the entire history of the patients that had these injuries
            injury_mask_series = True
            for cn in groupby_columns: injury_mask_series &= (frvrs_logs_df[cn] == eval(cn))
            injury_mask_series &= (
                (frvrs_logs_df.injury_record_patient_id == patient_id) |
                (frvrs_logs_df.injury_treated_patient_id == patient_id) |
                (frvrs_logs_df.tool_applied_patient_id == patient_id)
            )
            
            # Add the entire history of these injuries
            injury_ids_list = patient_df[bleeding_mask_series].injury_treated_id.tolist()
            injury_mask_series &= (
                frvrs_logs_df.injury_record_id.isin(injury_ids_list) |
                frvrs_logs_df.injury_treated_id.isin(injury_ids_list) |
                frvrs_logs_df.injury_id.isnull()
            )
            # injury_mask_series &= frvrs_logs_df.action_type.isin(['INJURY_RECORD', 'TOOL_APPLIED', 'INJURY_TREATED'])
            
            injury_records_df = frvrs_logs_df[injury_mask_series]
            # for action_type in injury_records_df.action_type: action_types_set.add(action_type)

            # Was the correct tool applied?
            mask_series = injury_records_df.tool_applied_sender.isin(['AppliedTourniquet', 'AppliedPackingGauze'])
            if injury_records_df[mask_series].shape[0]:
                row_dict['is_correct_tool_applied'] = True
                
                # Get the last tourniquet applied
                time_stop = injury_records_df[mask_series].elapsed_time.max()
                row_dict['tool_applied_time'] = time_stop
                
                # Get the time elapsed
                row_dict['run_tool_control_time'] = time_stop - time_start
                row_dict['engaged_tool_control_time'] = time_stop - patient_engaged
            
            else: row_dict['is_correct_tool_applied'] = False
            
            # Was the bleeding treated?
            mask_series = (injury_records_df.injury_treated_injury_treated == True) & (injury_records_df.injury_treated_injury_treated_with_wrong_treatment == False)
            if injury_records_df[mask_series].shape[0]:
                row_dict['is_bleeding_treated'] = True
                
                # Get the last tourniquet applied
                time_stop = injury_records_df[mask_series].elapsed_time.max()
                row_dict['bleeding_treated_time'] = time_stop
                
                # Get the time elapsed
                row_dict['run_bleeding_control_time'] = time_stop - time_start
                row_dict['engaged_bleeding_control_time'] = time_stop - patient_engaged
            
            # elif (row_dict['is_correct_tool_applied'] == True): 
            #     columns_list = ['injury_treated_injury_treated', 'tool_applied_sender']
            #     idx_list = injury_records_df[columns_list].drop_duplicates().index.tolist()
            #     mask_series = frvrs_logs_df.index.isin(idx_list)
            #     columns_list = sorted([
            #         'injury_treated_required_procedure', 'injury_record_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'injury_record_id',
            #         'injury_treated_id', 'tool_applied_sender', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'elapsed_time',
            #         'action_type', 'file_name'
            #     ])
            #     display(frvrs_logs_df[mask_series][columns_list].sort_values('elapsed_time').T)
            #     raise
            else: row_dict['is_bleeding_treated'] = False
        
        else:
            row_dict['is_patient_bleeding'] = False
            row_dict['is_correct_tool_applied'] = False
            row_dict['is_bleeding_treated'] = False
    
    rows_list.append(row_dict)
run_control_time_df = DataFrame(rows_list)

In [20]:

base_mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False) & (frvrs_logs_df.is_a_one_triage_file == True)
mask_series = base_mask_series & frvrs_logs_df.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
groupby_columns = ['session_uuid', 'scene_index']
total_bleeder_scenes_count = len(frvrs_logs_df[mask_series].groupby(groupby_columns).groups)
total_bleeders_count = len(frvrs_logs_df[mask_series].groupby(groupby_columns+['patient_id']).groups)
print(f'{total_bleeders_count:,} total bleeders in {total_bleeder_scenes_count} total scenes.')

1,471 total bleeders in 371 total scenes.


In [16]:

print(
    f"Of the one-per-file triage scenes that have tourniquet or woundpack as injury treated required procedures in them ({bleeder_scene_count}"
    f" scenes out of a total of {total_scene_count}), here is the bleeding/treated patient count:"
)

Of the one-per-file triage scenes that have tourniquet or woundpack as injury treated required procedures in them (371 scenes out of a total of 389), here is the bleeding/treated patient count:


In [17]:

groupby_columns = ['is_patient_bleeding', 'is_correct_tool_applied', 'is_bleeding_treated']
df = run_control_time_df.groupby(groupby_columns).size().reset_index(drop=False).rename(columns={0: 'patient_count'})
display(df.sort_values([
    'patient_count', 'is_patient_bleeding', 'is_correct_tool_applied', 'is_bleeding_treated'
], ascending=[False, True, True, True]))

,is_patient_bleeding,is_correct_tool_applied,is_bleeding_treated,patient_count
2,True,True,True,287
1,True,True,False,57
0,False,False,False,27


In [60]:

run_control_time_df.columns.tolist()

['session_uuid', 'scene_index', 'scene_start', 'patient_id', 'is_patient_bleeding', 'is_correct_tool_applied', 'is_bleeding_treated', 'patient_engagement_time', 'tool_applied_time', 'run_tool_control_time', 'engaged_tool_control_time']

In [9]:

columns_list = ['run_control_time']
df = run_control_time_df.groupby(groupby_columns)[columns_list].max()
nu.show_time_statistics(df, columns_list)

,mean,mode,median,SD,min,25%,50%,75%,max
run_control_time,3:56,25 sec,3:51,±1:44,25 sec,2:41,3:51,4:52,10:49



    c.  Calculate hemorrhage control per patient defined as “time patient engaged to time tool applied (tourniquet) for that patient”

        1.  Median = 11 sec, SD = ±47 sec, IQR = 6 sec, 20 sec)

In [30]:

columns_list = ['engaged_control_time']
df = run_control_time_df.groupby(groupby_columns)[columns_list].mean()
nu.show_time_statistics(df, columns_list)

,mean,mode,median,SD,min,25%,50%,75%,max
engaged_control_time,22 sec,7 sec,11 sec,±47 sec,-1:55,6 sec,11 sec,20 sec,7:17



3.  **Calculate the proportion of responders that consistently evaluated patients appropriately in this order: still, able to wave, and able to walk.**

    a.  Responders who evaluated patients in correct order: **<u data-renderer-mark="true">6.94%</u>**

In [18]:

columns_list = ['scene_type', 'is_scene_aborted', 'is_a_one_triage_file']
frvrs_logs_df[columns_list].drop_duplicates().sort_values(columns_list)

,scene_type,is_scene_aborted,is_a_one_triage_file
131522,Orientation,False,False
0,Orientation,False,True
71303,Triage,False,False
228,Triage,False,True


In [15]:

import numpy as np

engagment_sort_df = nu.load_object('engagment_sort_df')
groupby_columns = ['session_uuid', 'scene_index']
rows_list = []
idx_list = []
base_mask_series = (engagment_sort_df.scene_type == 'Triage') & (engagment_sort_df.is_scene_aborted == False) & (engagment_sort_df.is_a_one_triage_file == True)
for (session_uuid, scene_index), scene_df in engagment_sort_df[base_mask_series].groupby(groupby_columns):
    row_dict = {}
    for cn in groupby_columns: row_dict[cn] = eval(cn)
        
    mask_series = (scene_df.rsquared_adj == 1.0)
    count_correct = scene_df[mask_series].shape[0]
    row_dict['count_correct'] = count_correct
    
    mask_series = (scene_df.rsquared_adj < 1.0)
    count_incorrect = scene_df[mask_series].shape[0]
    row_dict['count_incorrect'] = count_incorrect

    total_count = count_correct + count_incorrect
    row_dict['total_count'] = total_count
    
    try: percentage_correct = 100*count_correct/total_count
    except ZeroDivisionError:
        percentage_correct = np.nan
        idx_list.append(scene_df.index.tolist()[0])
    row_dict['percentage_correct'] = percentage_correct
    
    rows_list.append(row_dict)
percentage_correct_sort_df = DataFrame(rows_list)

In [27]:

mask_series = engagment_sort_df.index.isin(idx_list)
engagment_sort_df[mask_series]

,session_uuid,scene_index,logger_version,is_scene_aborted,scene_type,is_a_one_triage_file,last_threat_engaged,last_still_engaged,last_walker_engaged,last_waver_engaged,rsquared_adj
32,0950437e-94b0-45bf-8d11-dd446fd0e4a2,0,1.0,False,Triage,True,NaN,NaN,68436.0,NaN,NaN
55,1066671d-2a1d-4744-b66f-e4b48548701f,0,1.3,False,Triage,True,NaN,NaN,NaN,NaN,NaN
69,158e6365-673b-4030-8b36-6704be5996a2,0,1.0,False,Triage,True,NaN,NaN,NaN,NaN,NaN
104,2310f107-d9d2-418e-a2d7-dd7a17924544,0,1.0,False,Triage,True,NaN,NaN,NaN,NaN,NaN
233,5325cdbf-d627-4aba-b5b4-9a62240d599f,0,1.0,False,Triage,True,NaN,64732.0,38403.0,NaN,NaN
234,54aaf31a-22bc-46f2-a810-8564161bf8d0,0,1.0,False,Triage,True,NaN,NaN,NaN,NaN,NaN
242,5c2a444a-9c8d-4c65-bef2-c5f47f6b258d,0,1.0,False,Triage,True,NaN,NaN,NaN,68077.0,NaN
249,5da6af4d-f3bc-4e05-9c5b-cb7d5aa68e7e,1,1.3,False,Triage,True,NaN,97371.0,NaN,104616.0,NaN
342,7c2549d4-97a4-4389-bd03-029396714f59,0,1.3,False,Triage,True,NaN,NaN,NaN,NaN,NaN
392,8ec8afba-8533-4915-898f-5769c1258c61,0,1.3,False,Triage,True,NaN,241847.0,NaN,NaN,NaN


In [16]:

columns_list = ['total_count', 'percentage_correct']
nu.get_minority_combinations(percentage_correct_sort_df, columns_list)

,session_uuid,scene_index,count_correct,count_incorrect,total_count,percentage_correct
261,b0ff9d39-598b-46b3-b3bc-1be3a791f10c,1,1,0,1,100.0
94,3ce991bc-a2d1-46fd-a3a4-d3611e946ed7,1,0,1,1,0.0


In [18]:

ave_percentage_correct = percentage_correct_sort_df.percentage_correct.mean()#, 100*df.count_correct.sum()/df.total_count.sum()
print(
    f'Responders who evaluated patients in correct order: {ave_percentage_correct:.2f}%'
    f' (100*{percentage_correct_sort_df.count_correct.sum()}/{percentage_correct_sort_df.total_count.sum()};'
    f' {len(responders_list)-percentage_correct_sort_df.total_count.sum()}'
    f' of the {len(responders_list)} did not have patients with patient_engaged_sort designations).'
    )

Responders who evaluated patients in correct order: 16.89% (100*63/373; 16 of the 389 did not have patients with patient_engaged_sort designations).



## Jacob's Abstract Edits

In [33]:

mask_series = (triage_count_df.triage_count == 1)
responders_list = triage_count_df[mask_series].file_name.tolist()

mask_series = frvrs_logs_df.file_name.isin(responders_list) & (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False)
ave_patients_count = frvrs_logs_df[mask_series].groupby(['session_uuid', 'scene_index']).patient_id.nunique().mean()

columns_list = ['run_control_time']
df = run_control_time_df.groupby(groupby_columns)[columns_list].max()
tttt_dict = nu.get_statistics(df, columns_list).applymap(lambda x: nu.format_timedelta(timedelta(milliseconds=int(x))).replace('sec', 'seconds').replace(
    'min', 'minutes'
), na_action='ignore').to_dict()['run_control_time']

ave_percentage_correct = percentage_correct_sort_df.percentage_correct.mean()

columns_list = ['triage_time']
tt_dict = nu.get_statistics(run_time_df, columns_list).applymap(lambda x: nu.format_timedelta(timedelta(milliseconds=int(x))).replace('sec', 'seconds').replace(
    'min', 'minutes'
), na_action='ignore').to_dict()['triage_time']

columns_list = ['engaged_control_time']
df = run_control_time_df.groupby(groupby_columns)[columns_list].mean()

columns_list = ['engaged_control_time']
df = run_control_time_df.groupby(groupby_columns)[columns_list].mean()
hcpp_dict = nu.get_statistics(df, columns_list).applymap(lambda x: nu.format_timedelta(timedelta(milliseconds=int(x))).replace('sec', 'seconds').replace(
    'min', 'minutes'
), na_action='ignore').to_dict()['engaged_control_time']

results_str = f"""RESULTS: A total of {len(responders_list)} participants triaged an average of {ave_patients_count:.0f} patients each in the VR First Responder simulation. Median total time to triage the scene was {tt_dict['median']} (SD = {tt_dict['SD']}, IQR = {tt_dict['25%']}, {tt_dict['75%']}). Through their sorting process, {ave_percentage_correct:.0f}% of clinicians consistently evaluated patients appropriately in this order: still, able to wave, and able to walk. Median time to life-threatening hemorrhage control was {tttt_dict['median']} (SD = {tttt_dict['SD']}, IQR = {tttt_dict['25%']}, {tttt_dict['75%']}). Median time for hemorrhage control per patient was low at {hcpp_dict['median']} (SD = {hcpp_dict['SD']}, IQR = {hcpp_dict['25%']}, {hcpp_dict['75%']})."""
print(results_str)

RESULTS: A total of 389 participants triaged an average of 11 patients each in the VR First Responder simulation. Median total time to triage the scene was 7:38 (SD = 2:27, IQR = 6:13, 8:59). Through their sorting process, 17% of clinicians consistently evaluated patients appropriately in this order: still, able to wave, and able to walk. Median time to life-threatening hemorrhage control was 3:51 (SD = 1:44, IQR = 2:41, 4:52). Median time for hemorrhage control per patient was low at 11 seconds (SD = 47 seconds, IQR = 6 seconds, 20 seconds).
